In [ ]:
import pandas as pd 
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline

from gensim import corpora, models, similarities, matutils
from gensim.models import Word2Vec
from nltk.corpus import gutenberg

# keras
np.random.seed(13)
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Activation, SimpleRNN, GRU, LSTM, Convolution1D, MaxPooling1D, Add, Dropout
from IPython.display import SVG
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

import tensorflow as tf
# import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
from keras import Model
from keras.layers import Dense

from keras_pandas.Automater import Automater

In [ ]:
#Reading in dataset of FFT data and sex
df_fft = pd.read_csv('full_dataset_sex')
df_fft.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
#Binarizing the Gender columns to make male = 1 and female = 0
df_fft['Sex'] = [1 if x == 'm' else 0 for x in df_fft['Gender']]
df_fft.to_csv('full_dataset_sex')

In [ ]:
#Averaging the first 25 datapoints for each speaker
df_mean = df_fft.groupby('ID').head(25)
df_mean = df_mean.groupby('ID').mean()

In [ ]:
#Creaing X and y variables
X_features = df_mean.iloc[:,:190]
y_target = df_mean['Sex'].values

In [ ]:
#Creating column names for Keras-Pandas
column_names = list(range(190))
column_names = column_names + ['Spectral_Centroid', 'Spectral_Centroid_std', 'Skew', 'Kurtosis',
       'RMSE', 'RMSE_std', 'Spectral_rolloff', 'Tempo', 'mfcc_1', 'mfcc_2',
       'mfcc_3', 'Zero_Crossing_Rate', 'Zero_Crossing_std']

In [ ]:
#Turning them into strings
for count, i in enumerate(column_names):
    column_names[count] = str(i)

In [ ]:
#Running a dense neural network using Keras-Pandas Library
with tf.device('/gpu:0'):
    observations = df_fft

    # Transform the data set, using keras_pandas
    numerical_vars = column_names
    categorical_vars = ['Sex']

    auto = Automater(numerical_vars=numerical_vars, categorical_vars=categorical_vars, response_var='Sex')
    X, y = auto.fit_transform(observations)

    print(y)
    # Start model with provided input nub
    x = auto.input_nub

    # Fill in your own hidden layers
    x = Dense(256)(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128)(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(128)(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64)(x)
    x = Dense(64, activation='relu')(x)
    



    # End model with provided output nub
    x = auto.output_nub(x)

    model = Model(inputs=auto.input_layers, outputs=x)
    model.compile(optimizer='Adam', loss=auto.loss, metrics=['accuracy'])

    # Train model
    model.fit(X, y, epochs=5, validation_split=.2)